## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random

<IPython.core.display.Javascript object>

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# answer goes here

student_df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
student_df.head()


,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5


<IPython.core.display.Javascript object>

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# answer goes here

student_df_q5 = student_df.filter(like='Q5')
X = student_df_q5.copy()
X


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0


<IPython.core.display.Javascript object>

The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
# answer goes here

dists = pdist(X, metric='hamming')
X_dists = pd.DataFrame(squareform(dists))
X_dists



,0,1,2,3,4,5,6,7,8,9,...,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,...,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,...,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,...,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2953,0.2,0.2,0.3,0.6,0.3,0.3,0.2,0.5,0.4,0.4,...,0.1,0.1,0.4,0.3,0.5,0.0,0.2,0.2,0.4,0.2
2954,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2955,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2956,0.2,0.2,0.1,0.6,0.5,0.3,0.2,0.3,0.6,0.4,...,0.3,0.5,0.4,0.5,0.5,0.4,0.2,0.2,0.0,0.2


<IPython.core.display.Javascript object>

Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
# answer goes here

num_clusters = 2

initial_medoids = X.sample(num_clusters).index

kmedoids_instance = kmedoids(X_dists, initial_medoids, data_type="distance_matrix")

kmedoids_instance.process()

medoids = kmedoids_instance.get_medoids()
X.loc[medoids].style.background_gradient()


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1835,0,1,1,0,0,0,0,1,0,0


<IPython.core.display.Javascript object>

In [7]:
clusters = kmedoids_instance.get_clusters()
X["cluster"] = 0
for i in range(len(clusters)):
    X.loc[clusters[i], "cluster"] = i
X

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0
3,1,1,0,1,1,1,1,0,1,0,0
4,0,1,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0,0


<IPython.core.display.Javascript object>

If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial medoids that are far apart. Re-run the previous process, except with a random pair of medoids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [8]:
# answer goes here

X_dists["id"] = X_dists.index

df = pd.melt(X_dists, id_vars="id", value_vars=X_dists.columns[X_dists.columns != "id"])

df.columns = ['response_id_1', 'response_id_2', 'dissimilarity'] 
df



,response_id_1,response_id_2,dissimilarity
0,0,0,0.0
1,1,0,0.0
2,2,0,0.1
3,3,0,0.6
4,4,0,0.3
...,...,...,...
8749759,2953,2957,0.2
8749760,2954,2957,0.2
8749761,2955,2957,0.2
8749762,2956,2957,0.2


<IPython.core.display.Javascript object>

In [9]:
df2 = df.loc[lambda x: x["dissimilarity"] >= 0.8]
df2

,response_id_1,response_id_2,dissimilarity
245,245,0,0.8
263,263,0,0.8
496,496,0,0.8
752,752,0,0.8
812,812,0,0.8
...,...,...,...
8743789,2899,2955,0.8
8744952,1104,2956,0.8
8745581,1733,2956,0.8
8746809,3,2957,0.8


<IPython.core.display.Javascript object>

In [10]:
df2.sort_values(by="dissimilarity", ascending=False)

,response_id_1,response_id_2,dissimilarity
725051,341,245,1.0
1468897,1729,496,1.0
3819359,581,1291,1.0
2716726,1282,918,1.0
2716735,1291,918,1.0
...,...,...,...
2814498,1440,951,0.8
2814558,1500,951,0.8
2814577,1519,951,0.8
2814584,1526,951,0.8


<IPython.core.display.Javascript object>

In [11]:
sample = df2.sample(1)
sample

,response_id_1,response_id_2,dissimilarity
7533559,2491,2546,0.8


<IPython.core.display.Javascript object>

In [12]:
[sample.response_id_1.values[0], sample.response_id_2.values[0]]

[2491, 2546]

<IPython.core.display.Javascript object>

In [20]:
sample = df2.sample(1)
print(sample)

initial_medoids = [sample.response_id_1.values[0], sample.response_id_2.values[0]]

kmedoids_instance = kmedoids(X_dists, initial_medoids, data_type="distance_matrix")

kmedoids_instance.process()

medoids = kmedoids_instance.get_medoids()
X.loc[medoids].style.background_gradient()

        response_id_1 response_id_2  dissimilarity
726847           2137           245            0.8


,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
2137,0,1,0,0,0,0,0,0,0,0,0
245,1,1,1,1,1,1,1,1,1,0,1


<IPython.core.display.Javascript object>

In [21]:
# The results do seem to be more stable. Most of the resulting medoids have only one positive
# response in common

<IPython.core.display.Javascript object>

In [22]:
clusters = kmedoids_instance.get_clusters()
X["cluster"] = 0
for i in range(len(clusters)):
    X.loc[clusters[i], "cluster"] = i
X

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,cluster
0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0
3,1,1,0,1,1,1,1,0,1,0,1
4,0,1,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0,0


<IPython.core.display.Javascript object>